In [1]:
DATA_NAME = 'syntheticMRI2D-coronal' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_horizontal_wavelet_coronal_syntheticMRI2D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,3,4,5,6,7,8,9
3,1192180.58,47.24,-0.66,-8.33,0.68,-0.42,0.01
4,47.24,1616104.81,31.40,5.01,7.00,-1.22,-0.04
5,-0.66,31.40,448837.04,1.97,-1.66,0.89,0.02
6,-8.33,5.01,1.97,110906.65,-0.89,-0.20,0.09
7,0.68,7.00,-1.66,-0.89,17815.93,0.08,0.00
8,-0.42,-1.22,0.89,-0.20,0.08,1525.25,-0.00
9,0.01,-0.04,0.02,0.09,0.00,-0.00,72.96


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

,3,4,5,6,7,8,9
3,1.00000,0.00003,-0.00000,-0.00002,0.00000,-0.00001,0.00000
4,0.00003,1.00000,0.00004,0.00001,0.00004,-0.00002,-0.00000
5,-0.00000,0.00004,1.00000,0.00001,-0.00002,0.00003,0.00000
6,-0.00002,0.00001,0.00001,1.00000,-0.00002,-0.00002,0.00003
7,0.00000,0.00004,-0.00002,-0.00002,1.00000,0.00002,0.00000
8,-0.00001,-0.00002,0.00003,-0.00002,0.00002,1.00000,-0.00000
9,0.00000,-0.00000,0.00000,0.00003,0.00000,-0.00000,1.00000


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

82.12193398053806

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.85234869e+11 1.93209467e+11 2.67135085e+10 1.53471270e+09
 3.52222727e+07 1.93946253e+05 3.33488154e-29]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,-0.241218,0.969675,-0.038306,-0.008688,-0.001385,-0.000120,-0.000006
1,0.966352,0.236271,-0.099535,-0.020547,-0.003263,-0.000280,-0.000013
2,0.086039,0.059969,0.992092,-0.068203,-0.010139,-0.000864,-0.000041
3,0.023449,0.017236,0.064817,0.995909,-0.055649,-0.004602,-0.000219
4,0.004982,0.003670,0.013249,0.054492,0.997464,-0.043333,-0.002050
5,0.000638,0.000471,0.001693,0.006862,0.042851,0.997907,-0.047914
6,0.000061,0.000045,0.000162,0.000656,0.004090,0.047779,0.998849


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[1.241218013841736,
 0.7637294029169832,
 0.007908423365063477,
 0.004091474918889171,
 0.002535765564893322,
 0.002093488275858424,
 0.0011506721356453475]